In [26]:
import numpy as np
import pandas as pd
import os
import collections
import spacy
import sys
sys.path.append("..")

import re
import time
import scispacy
from tqdm import tqdm
from heuristic_tokenize import sent_tokenize_rules 
from spacy.language import Language
from utils.tools import df_analyze

nlp = spacy.load("en_core_sci_md")

In [14]:
root = '../Mimic_ts/rawdata/'
df = pd.read_csv(os.path.join(root, 'Noteevents.csv'),low_memory=False)

In [3]:
df.head(5)

,Unnamed: 0,PatientID,RecordID,Text,Time,category
0,0,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...,5730400800,Discharge summary
1,1,13702,107527.0,Admission Date: [**2118-6-2**] Discharg...,4684615200,Discharge summary
2,2,13702,167118.0,Admission Date: [**2119-5-4**] D...,4714423200,Discharge summary
3,3,13702,196489.0,Admission Date: [**2124-7-21**] ...,4879620000,Discharge summary
4,4,26880,135453.0,Admission Date: [**2162-3-3**] D...,6066151200,Discharge summary


In [12]:
df_analyze(df)

,Column,null,unique,type
0,PatientID,0,46146,int64
1,RecordID,231836,58361,float64
2,Text,0,2023185,object
3,Time,0,1487486,int64
4,category,0,15,object


In [15]:
df.dropna(inplace=True)
df.drop(labels='Unnamed: 0', axis=1, inplace=True)
df['RecordID'] = df['RecordID'].apply(lambda x: int(x))

In [16]:
df.head()

,PatientID,RecordID,Text,Time,category
0,22532,167853,Admission Date: [**2151-7-16**] Dischar...,5730400800,Discharge summary
1,13702,107527,Admission Date: [**2118-6-2**] Discharg...,4684615200,Discharge summary
2,13702,167118,Admission Date: [**2119-5-4**] D...,4714423200,Discharge summary
3,13702,196489,Admission Date: [**2124-7-21**] ...,4879620000,Discharge summary
4,26880,135453,Admission Date: [**2162-3-3**] D...,6066151200,Discharge summary


In [19]:
text = df.iloc[0].Text

In [20]:
doc = nlp(text)

In [23]:
a = list(doc.sents)

In [24]:
a[0]

Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]


Service:
ADDENDUM:

RADIOLOGIC STUDIES:  Radiologic studies also included a chest
CT, which confirmed cavitary lesions in the left lung apex
consistent with infectious process/tuberculosis.

In [29]:
str(a[2]).split()

['HEAD',
 'CT:',
 'Head',
 'CT',
 'showed',
 'no',
 'intracranial',
 'hemorrhage',
 'or',
 'mass',
 'effect,',
 'but',
 'old',
 'infarction',
 'consistent',
 'with',
 'past',
 'medical',
 'history.']

In [10]:
df.CATEGORY.unique()

array(['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ',
       'Rehab Services', 'Case Management ', 'Respiratory ', 'Nutrition',
       'General', 'Social Work', 'Pharmacy', 'Consult', 'Radiology',
       'Nursing/other'], dtype=object)

In [10]:
df0 = df[df['DESCRIPTION']=='Report']

In [14]:
df1 = df[['SUBJECT_ID','CHARTDATE']]

In [16]:
test_p = df1[df1['SUBJECT_ID']==26880]
print(test_p)

         SUBJECT_ID   CHARTDATE
4             26880  2162-03-25
100978        26880  2162-03-12
100979        26880  2162-03-05
100980        26880  2162-03-03
101480        26880  2162-03-16
101481        26880  2162-03-13
987980        26880  2162-03-06
987988        26880  2162-03-11
989020        26880  2162-03-11
989907        26880  2162-03-03
990633        26880  2162-03-14
999169        26880  2162-03-13
999362        26880  2162-03-14
999732        26880  2162-03-05
1000692       26880  2162-03-03
1000854       26880  2162-03-03
1000890       26880  2162-03-09
1001024       26880  2162-03-03
1001025       26880  2162-03-03
1001026       26880  2162-03-03
1001431       26880  2162-03-10
1001621       26880  2162-03-04
1001738       26880  2162-03-16
1004585       26880  2162-03-18
1599696       26880  2162-03-09
1599952       26880  2162-03-07
1600235       26880  2162-03-07
1600236       26880  2162-03-08
1600237       26880  2162-03-08
1600238       26880  2162-03-09
1601551 

In [7]:
patients = list(df0['SUBJECT_ID'])

In [8]:
def filtering(nums, l, r):
    mymap = collections.defaultdict(int)
    length = len(nums)
    for i in range(length):
        mymap[nums[i]] += 1
    res = []
    for key, value in mymap.items():
        if l < value < r:
            res.append(key)
    return res

In [9]:
outs = filtering(patients, 20, 30)
print(len(outs))
print(outs)

3202
[26880, 79900, 9805, 22663, 23224, 14291, 82208, 24470, 18363, 9801, 14330, 827, 25198, 1428, 5508, 9599, 22667, 24338, 5051, 10331, 27869, 83596, 31724, 71184, 11202, 28488, 11327, 16044, 32139, 21892, 22432, 29919, 9539, 26936, 4826, 68867, 23693, 18618, 28068, 10145, 22901, 151, 18477, 15028, 24582, 7934, 9428, 72270, 12145, 2712, 23787, 31671, 28578, 19300, 90755, 12110, 17905, 79556, 3403, 50093, 16148, 15918, 2034, 22944, 25590, 105, 17024, 1937, 14031, 28077, 5783, 13739, 29198, 27696, 32559, 32219, 3362, 73648, 1044, 24718, 32013, 5564, 22354, 4316, 16242, 18291, 42468, 268, 12732, 867, 17451, 14756, 59411, 22503, 11997, 24242, 96261, 96108, 75300, 80891, 1811, 28857, 7066, 90688, 6615, 75741, 88857, 3266, 11312, 29581, 25201, 26937, 14885, 21085, 8312, 44586, 58319, 10655, 30460, 7468, 12795, 2819, 18139, 65759, 28267, 20465, 4535, 12472, 3436, 24674, 81864, 19125, 13794, 93774, 6541, 10832, 14755, 32412, 18309, 83932, 9611, 947, 85699, 13110, 18831, 53787, 1344, 32020, 6

In [10]:
df1 = df0[df0['SUBJECT_ID']==20279]

In [11]:
df1.iloc[0].DESCRIPTION == 'Report'

True

In [12]:
print(df1)

          ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE            CHARTTIME  \
54724      50088       20279  156922.0  2125-11-05                  NaN   
54725      50089       20279  156922.0  2125-11-23                  NaN   
99141      99689       20279  156922.0  2125-11-19                  NaN   
99155      99790       20279  156922.0  2125-11-06                  NaN   
270912    283993       20279       NaN  2125-11-26                  NaN   
306300    284229       20279  156922.0  2125-10-26                  NaN   
306301    284230       20279       NaN  2125-09-27                  NaN   
306302    284231       20279       NaN  2125-09-27                  NaN   
1496908  1529276       20279  156922.0  2125-11-01  2125-11-01 04:37:00   
1499622  1529269       20279  156922.0  2125-10-29  2125-10-29 17:53:00   
1499623  1529270       20279  156922.0  2125-10-30  2125-10-30 05:45:00   
1499624  1529271       20279  156922.0  2125-10-30  2125-10-30 06:01:00   
1499625  1529272       20

In [15]:

for i in range(20):
    print('--------------------------------------------')
    print(df1.iloc[i]['TEXT'])
    print('--------------------------------------------')

#print(df1.iloc[0]['TEXT'])

--------------------------------------------
Admission Date:  [**2125-10-26**]       Discharge Date:  [**2125-11-5**]

Date of Birth:   [**2077-6-28**]       Sex:  M

Service:  [**Location (un) 259**]/MEDICINE

HISTORY OF PRESENT ILLNESS:  The patient is a 48 year-old
male with HIV/AIDS, end stage renal disease on hemodialysis
who presented with rectal bleeding.  The patient had recently
been discharged from [**Hospital1 69**] on
[**10-17**] when he left against medical advise.  During that
admission he had been treated for a gout flare sacral
decubitus wound debridement and had a negative workup for
abdominal pain and diarrhea.  At this time the patient had
three to four days of bright red blood per rectum.  He denies
abdominal pain.  Denies nausea or vomiting.  He also denies
fever or chills.  The bleeding had been brisk for three to
four days since it began and then subsided on the day prior
to admission, however, on the day of admission he had
increased amounts of bright red blood 

In [ ]:
len(df)

In [ ]:
2083180/46146

In [ ]:
print(df.iloc[4].TEXT)